In [ ]:
!nvidia-smi

Thu May 19 08:22:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q transformers

In [ ]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer
from transformers import RobertaModel,RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/aug_clean_train.csv', header=0)
df.head()

,text_clean,Sentiment
0,it s a confusing odd time for the shopping pub...,Negative
1,in 2019 d2c ecommerce sales reached 1428 billi...,Positive
2,chinese residents are paying exorbitant prices...,Negative
3,list of supermarkets grocery shops and vegetab...,Neutral
4,its there any wonder tesco and other supermark...,Positive


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv', header=0)
df_test.head()

,text_clean,Sentiment
0,we may not have any toilet paper in our house ...,Positive
1,really whats the downside of coronavirus for a...,Positive
2,hello everyone we made amp sell high quality m...,Extremely Positive
3,happy to report that i jumped on the panic sho...,Positive
4,just been to the supermarket why do all women ...,Neutral


# Sentiment column ananlysis

In [ ]:
df['Sentiment'].value_counts()

Positive              11381
Negative               9889
Neutral                7560
Extremely Positive     6618
Extremely Negative     5475
Name: Sentiment, dtype: int64

In [ ]:
df['Sentiment'] = df['Sentiment'].map({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                                       'Positive': 2, 'Extremely Positive': 2})

In [ ]:
df_test['Sentiment'] = df_test['Sentiment'].map({'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                                                 'Positive': 2, 'Extremely Positive': 2})

In [ ]:
df['Sentiment'].value_counts()

2    17999
0    15364
1     7560
Name: Sentiment, dtype: int64

## Class Balanceing by RandomOverSampler

In [ ]:
# ros = RandomOverSampler()
# train_x, train_y = ros.fit_resample(np.array(df['text_clean']).reshape(-1, 1), np.array(df['Sentiment']).reshape(-1, 1))
# train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns=['text_clean', 'Sentiment'])

In [ ]:
train_os = df

In [ ]:
train_os['Sentiment'].value_counts()

2    17999
0    15364
1     7560
Name: Sentiment, dtype: int64

## Train - Validation - Test split

In [ ]:
train_texts = train_os['text_clean'].values.tolist()
train_labels = train_os['Sentiment'].values.tolist()

In [ ]:
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, stratify=train_labels, random_state=42)

In [ ]:
test_texts = df_test['text_clean'].values.tolist()
test_labels = df_test['Sentiment'].values.tolist()

In [ ]:
print(f"TRAINING DATA: {len(train_texts)} \nTESTING DATA: {len(test_texts)}" )

TRAINING DATA: 40923 
TESTING DATA: 3787


# Robert MLM

In [ ]:
MAX_LEN=128

In [ ]:
import torch
from tqdm.auto import tqdm
from transformers import AdamW
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_encodings = tokenizer(train_texts, max_length=MAX_LEN,
                            padding='max_length', truncation=True,
                            return_tensors="pt")
test_encodings = tokenizer(test_texts, max_length=MAX_LEN,
                           padding='max_length', truncation=True,
                           return_tensors="pt")

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
train_encodings['labels'] = train_encodings['input_ids'].detach().clone()
test_encodings['labels'] = test_encodings['input_ids'].detach().clone()

## Mask the input_ids

In [ ]:
train_random_tensor = torch.rand(train_encodings['input_ids'].shape)
test_random_tensor = torch.rand(test_encodings['input_ids'].shape)

In [ ]:
# creating a mask tensor of float values ranging from 0 to 1 and avoiding special tokens
train_masked_tensor = (train_random_tensor < 0.15)*(train_encodings['input_ids'] != 101)*(train_encodings['input_ids'] != 102)*(train_encodings['input_ids'] != 0)
test_masked_tensor = (test_random_tensor < 0.15)*(test_encodings['input_ids'] != 101)*(test_encodings['input_ids'] != 102)*(test_encodings['input_ids'] != 0)

In [ ]:
# getting all those indices from each row which are set to True, i.e. masked.
train_nonzeros_indices = []
for i in range(len(train_masked_tensor)):
    train_nonzeros_indices.append(torch.flatten(train_masked_tensor[i].nonzero()).tolist())

test_nonzeros_indices = []
for i in range(len(test_masked_tensor)):
    test_nonzeros_indices.append(torch.flatten(test_masked_tensor[i].nonzero()).tolist())

In [ ]:
# setting the values at those indices to be a MASK token (103) for every row in the original input_ids.
for i in range(len(train_encodings['input_ids'])):
    train_encodings['input_ids'][i, train_nonzeros_indices[i]] = 103

for i in range(len(test_encodings['input_ids'])):
    test_encodings['input_ids'][i, test_nonzeros_indices[i]] = 103

In [ ]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item
    
    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = TweetDataset(train_encodings)
test_dataset = TweetDataset(test_encodings)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False
)

# Fine-tuning with native PyTorch/TensorFlow

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
model.to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
epochs = 10
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.metrics import classification_report

def accuracy(input_ids, logits, labels):
    predicted = logits.argmax(axis=2)
    
    mask_predicted = []
    mask_label = []
    true_predicted = []

    for i, line in enumerate(input_ids == 103):
        for j, ele in enumerate(line):
            if ele == True:
                mask_predicted.append(predicted[i,j].item())
                mask_label.append(labels[i,j].item())

    for i in range(len(mask_predicted)):
        if mask_predicted[i] == mask_label[i]:
            true_predicted.append(1)
        else:
            true_predicted.append(0)

    acc = sum(true_predicted)/len(true_predicted)
    return acc

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loop = tqdm(train_dataloader)
    for batch in train_loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loop.set_description("Epoch {}:".format(epoch))
        train_loop.set_postfix(loss=loss.item())

    model.eval()
    test_loop = tqdm(test_dataloader)
    total_acc = []
    with torch.no_grad():
        for batch in test_loop:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            logits = outputs.logits

            test_loop.set_description("Epoch {}".format(epoch))
            test_loop.set_postfix(loss=loss.item())
            total_acc.append(accuracy(input_ids, logits, labels))
    total_acc = np.array(total_acc)
    print("Accuracy: {}".format(total_acc.mean()))

  0%|          | 0/640 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Accuracy: 0.4101858409416483


  0%|          | 0/640 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

...

  0%|          | 0/60 [00:00<?, ?it/s]

Accuracy: 0.48447469010529975


  0%|          | 0/640 [00:00<?, ?it/s]

  0%|          | 0/60 [00:00<?, ?it/s]

Accuracy: 0.48572906093190243
